## This demo shows how to use LangChain's SQLDatabaseChain with Llama2 to chat about structured data stored in a SQL DB.  
* As the 2023-24 NBA season is around the corner, we use the NBA roster info saved in a SQLite DB to show you how to ask Llama2 questions about your favorite teams or players. 
* Because the SQLDatabaseChain API implementation is still in the langchain_experimental package, you'll see more issues that come with using the cutting edge experimental features, and how we succeed resolving some of the issues but fail on some others.

In [ ]:
!pip install langchain replicate langchain_experimental

In [1]:
from langchain.llms import Replicate
from langchain.prompts import PromptTemplate
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [2]:
from getpass import getpass
import os

REPLICATE_API_TOKEN = getpass()
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

 ········


In [ ]:
llama2_13b_chat = "meta/llama-2-13b-chat:f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d"

# the reason we set system_prompt below is to ask Llama to generate only the SQL statement, instead of being wordy and adding something like
# "Sure! Here's the SQL query for the given input question: " before the SQL query; otherwise custom parsing will be needed.
llm = Replicate(
    model=llama2_13b_chat,
    model_kwargs={"temperature": 0.01, "top_p": 1, "max_new_tokens":500, "system_prompt": "Given an input question, convert it to a SQL query. No pre-amble."},
)

In [4]:
db = SQLDatabase.from_uri("sqlite:///nba_roster.db", sample_rows_in_table_info= 0)

# customize the default sqlite prompt defined in the link above
PROMPT_SUFFIX = """
Only use the following tables:
{table_info}

Question: {input}"""

# if return_sql is set to False, then calling db_chain.run will make two calls to llm: first get the SQL query statement, 
# then pass the result of running the SQL query along with the original question and the query to llm for NL answer
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, return_sql=False, 
                                     prompt=PromptTemplate(input_variables=["input", "table_info"], 
                                     template=PROMPT_SUFFIX))

In [5]:
# turn on the LangChain debug to see what exactly happens behind the scenes in terms of the inputs and outputs of calling Llama
import langchain
langchain.debug = True
answer = db_chain.run("How many unique teams are there?")

# Two Llama calls happen below 
# Note the difference of the first and second inputs for the Llama calls - before "llm:Replicate] Entering LLM run with input"
# About the output, for the first llm call, both "llm:Replicate] [2.10s] Exiting LLM run with output" and 
# "Exiting Chain run with output" return correctly the text of SQL statement "SELECT COUNT(DISTINCT Team) AS num_teams FROM nba_roster;"
# In the second llm call, "Exiting LLM run with output" also shows the correct result:
# "Sure! Here's the answer to your question:\n\nThere are 30 unique teams in the NBA."

[chain/start] [1:chain:SQLDatabaseChain] Entering Chain run with input:
{
  "query": "How many unique teams are there?"
}
[chain/start] [1:chain:SQLDatabaseChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "How many unique teams are there?\nSQLQuery:",
  "top_k": "5",
  "dialect": "sqlite",
  "table_info": "\nCREATE TABLE nba_roster (\n\t\"Team\" TEXT, \n\t\"NAME\" TEXT, \n\t\"Jersey\" TEXT, \n\t\"POS\" TEXT, \n\t\"AGE\" INTEGER, \n\t\"HT\" TEXT, \n\t\"WT\" TEXT, \n\t\"COLLEGE\" TEXT, \n\t\"SALARY\" TEXT\n)",
  "stop": [
    "\nSQLResult:"
  ]
}
[llm/start] [1:chain:SQLDatabaseChain > 2:chain:LLMChain > 3:llm:Replicate] Entering LLM run with input:
{
  "prompts": [
    "Only use the following tables:\n\nCREATE TABLE nba_roster (\n\t\"Team\" TEXT, \n\t\"NAME\" TEXT, \n\t\"Jersey\" TEXT, \n\t\"POS\" TEXT, \n\t\"AGE\" INTEGER, \n\t\"HT\" TEXT, \n\t\"WT\" TEXT, \n\t\"COLLEGE\" TEXT, \n\t\"SALARY\" TEXT\n)\n\nQuestion: How many unique teams are there?\nSQLQuery:"
  ]
}
[

In [6]:
# just show the answer without all the debug info 
print(answer)

Sure! Here's the answer to your question:

There are 30 unique teams in the NBA.


In [7]:
# try another question and start seeing why cutting edge may be inconsistent and cut and hurt 
answer = db_chain.run("Which team is Klay Thompson in?")

# the output of the first llm call is correct: SELECT Team FROM nba_roster WHERE NAME = 'Klay Thompson';"
# the second "Entering LLM run with input" also has correct input, containing "SQLResult: [('Golden State Warriors',)]"
# and yet the second "Exiting Chain run with output" insists generating the result:
# "Sure! Here's the SQL query for the given input question:\n\nSELECT Team FROM nba_roster WHERE NAME = 'Klay Thompson';"

[chain/start] [1:chain:SQLDatabaseChain] Entering Chain run with input:
{
  "query": "Which team is Klay Thompson in?"
}
[chain/start] [1:chain:SQLDatabaseChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Which team is Klay Thompson in?\nSQLQuery:",
  "top_k": "5",
  "dialect": "sqlite",
  "table_info": "\nCREATE TABLE nba_roster (\n\t\"Team\" TEXT, \n\t\"NAME\" TEXT, \n\t\"Jersey\" TEXT, \n\t\"POS\" TEXT, \n\t\"AGE\" INTEGER, \n\t\"HT\" TEXT, \n\t\"WT\" TEXT, \n\t\"COLLEGE\" TEXT, \n\t\"SALARY\" TEXT\n)",
  "stop": [
    "\nSQLResult:"
  ]
}
[llm/start] [1:chain:SQLDatabaseChain > 2:chain:LLMChain > 3:llm:Replicate] Entering LLM run with input:
{
  "prompts": [
    "Only use the following tables:\n\nCREATE TABLE nba_roster (\n\t\"Team\" TEXT, \n\t\"NAME\" TEXT, \n\t\"Jersey\" TEXT, \n\t\"POS\" TEXT, \n\t\"AGE\" INTEGER, \n\t\"HT\" TEXT, \n\t\"WT\" TEXT, \n\t\"COLLEGE\" TEXT, \n\t\"SALARY\" TEXT\n)\n\nQuestion: Which team is Klay Thompson in?\nSQLQuery:"
  ]
}
[llm

In [8]:
# just to see how unbelievable it truly is
print(answer)

Sure! Here's the SQL query for the given input question:

SELECT Team FROM nba_roster WHERE NAME = 'Klay Thompson';


In [9]:
# one way to fix this is to set return_sql to be True, so each db_chain.run call will only make one llm call, and we'll
# manually process each llm output
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, return_sql=True, 
                                     prompt=PromptTemplate(input_variables=["input", "table_info"], 
                                     template=PROMPT_SUFFIX))

In [10]:
# let's go back to the question that works before to make sure the change above doesn't break it
# because return_sql is True, and system_prompt is set to "Given an input question, convert it to a SQL query. No pre-amble." when 
# creating the llm instance, the db_chain's run will make just one Llama call and ask it to return the SQL statement only, 
# instead of making two Llama calls and returning the answer it can't do in the above case.
sql = db_chain.run("How many unique teams are there?")
print(sql)

[chain/start] [1:chain:SQLDatabaseChain] Entering Chain run with input:
{
  "query": "How many unique teams are there?"
}
[chain/start] [1:chain:SQLDatabaseChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "How many unique teams are there?\nSQLQuery:",
  "top_k": "5",
  "dialect": "sqlite",
  "table_info": "\nCREATE TABLE nba_roster (\n\t\"Team\" TEXT, \n\t\"NAME\" TEXT, \n\t\"Jersey\" TEXT, \n\t\"POS\" TEXT, \n\t\"AGE\" INTEGER, \n\t\"HT\" TEXT, \n\t\"WT\" TEXT, \n\t\"COLLEGE\" TEXT, \n\t\"SALARY\" TEXT\n)",
  "stop": [
    "\nSQLResult:"
  ]
}
[llm/start] [1:chain:SQLDatabaseChain > 2:chain:LLMChain > 3:llm:Replicate] Entering LLM run with input:
{
  "prompts": [
    "Only use the following tables:\n\nCREATE TABLE nba_roster (\n\t\"Team\" TEXT, \n\t\"NAME\" TEXT, \n\t\"Jersey\" TEXT, \n\t\"POS\" TEXT, \n\t\"AGE\" INTEGER, \n\t\"HT\" TEXT, \n\t\"WT\" TEXT, \n\t\"COLLEGE\" TEXT, \n\t\"SALARY\" TEXT\n)\n\nQuestion: How many unique teams are there?\nSQLQuery:"
  ]
}
[

In [11]:
print(db._execute(sql))
# the right answer is generated

[{'num_teams': 30}]


In [12]:
# let's try the query that fails
sql = db_chain.run("Which team is Klay Thompson in?")
print(sql)

[chain/start] [1:chain:SQLDatabaseChain] Entering Chain run with input:
{
  "query": "Which team is Klay Thompson in?"
}
[chain/start] [1:chain:SQLDatabaseChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Which team is Klay Thompson in?\nSQLQuery:",
  "top_k": "5",
  "dialect": "sqlite",
  "table_info": "\nCREATE TABLE nba_roster (\n\t\"Team\" TEXT, \n\t\"NAME\" TEXT, \n\t\"Jersey\" TEXT, \n\t\"POS\" TEXT, \n\t\"AGE\" INTEGER, \n\t\"HT\" TEXT, \n\t\"WT\" TEXT, \n\t\"COLLEGE\" TEXT, \n\t\"SALARY\" TEXT\n)",
  "stop": [
    "\nSQLResult:"
  ]
}
[llm/start] [1:chain:SQLDatabaseChain > 2:chain:LLMChain > 3:llm:Replicate] Entering LLM run with input:
{
  "prompts": [
    "Only use the following tables:\n\nCREATE TABLE nba_roster (\n\t\"Team\" TEXT, \n\t\"NAME\" TEXT, \n\t\"Jersey\" TEXT, \n\t\"POS\" TEXT, \n\t\"AGE\" INTEGER, \n\t\"HT\" TEXT, \n\t\"WT\" TEXT, \n\t\"COLLEGE\" TEXT, \n\t\"SALARY\" TEXT\n)\n\nQuestion: Which team is Klay Thompson in?\nSQLQuery:"
  ]
}
[llm

In [13]:
# the sql result looks good. let's run it to get the answer
print(db._execute(sql))

[{'Team': 'Golden State Warriors'}]


In [14]:
# how about a follow up question
sql = db_chain.run("What's his salary?")

[chain/start] [1:chain:SQLDatabaseChain] Entering Chain run with input:
{
  "query": "What's his salary?"
}
[chain/start] [1:chain:SQLDatabaseChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "What's his salary?\nSQLQuery:",
  "top_k": "5",
  "dialect": "sqlite",
  "table_info": "\nCREATE TABLE nba_roster (\n\t\"Team\" TEXT, \n\t\"NAME\" TEXT, \n\t\"Jersey\" TEXT, \n\t\"POS\" TEXT, \n\t\"AGE\" INTEGER, \n\t\"HT\" TEXT, \n\t\"WT\" TEXT, \n\t\"COLLEGE\" TEXT, \n\t\"SALARY\" TEXT\n)",
  "stop": [
    "\nSQLResult:"
  ]
}
[llm/start] [1:chain:SQLDatabaseChain > 2:chain:LLMChain > 3:llm:Replicate] Entering LLM run with input:
{
  "prompts": [
    "Only use the following tables:\n\nCREATE TABLE nba_roster (\n\t\"Team\" TEXT, \n\t\"NAME\" TEXT, \n\t\"Jersey\" TEXT, \n\t\"POS\" TEXT, \n\t\"AGE\" INTEGER, \n\t\"HT\" TEXT, \n\t\"WT\" TEXT, \n\t\"COLLEGE\" TEXT, \n\t\"SALARY\" TEXT\n)\n\nQuestion: What's his salary?\nSQLQuery:"
  ]
}
[llm/end] [1:chain:SQLDatabaseChain > 2:cha

In [15]:
# two issues here: the llm output is wordy again, even though we use the system prompt to ask it not to be; 
# the other issue makes sense: we didn't pass the context along with the follow-up to llm so it didn't know what "his" is.
print(sql)

Sure! Here is the SQL query based on the given question and table structure:

SELECT SALARY FROM nba_roster WHERE NAME = 'LeBron James';


In [16]:
# let's fix the first issue by custom parsing, which is typical in real LLM apps
# use regular expression to extract only the SQL SELECT statement
import re

pattern = r'SELECT[\s\S]*?;'
match = re.search(pattern, sql)
if match:
    sql = match.group()
    print(sql)
print(db._execute(sql))

SELECT SALARY FROM nba_roster WHERE NAME = 'LeBron James';
[{'SALARY': '$47,607,350'}]


In [17]:
# now let's try to fix the issue that the context (the previous question and answer) was not sent to LLM along with the new question
# SQLDatabaseChain.from_llm has a parameter "memory" which can be set to a ConversationBufferMemory instance, which looks promising.
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
db_chain_memory = SQLDatabaseChain.from_llm(llm, db, memory=memory, 
                                            verbose=True, return_sql=True, 
                                            prompt=PromptTemplate(input_variables=["input", "table_info"], 
                                            template=PROMPT_SUFFIX))

In [18]:
# use the db_chain_memory to run the original question again
question = "Which team is Klay Thompson in"
sql = db_chain_memory.run(question)
print(sql)

[chain/start] [1:chain:SQLDatabaseChain] Entering Chain run with input:
{
  "query": "Which team is Klay Thompson in",
  "history": ""
}
[chain/start] [1:chain:SQLDatabaseChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Which team is Klay Thompson in\nSQLQuery:",
  "top_k": "5",
  "dialect": "sqlite",
  "table_info": "\nCREATE TABLE nba_roster (\n\t\"Team\" TEXT, \n\t\"NAME\" TEXT, \n\t\"Jersey\" TEXT, \n\t\"POS\" TEXT, \n\t\"AGE\" INTEGER, \n\t\"HT\" TEXT, \n\t\"WT\" TEXT, \n\t\"COLLEGE\" TEXT, \n\t\"SALARY\" TEXT\n)",
  "stop": [
    "\nSQLResult:"
  ],
  "history": ""
}
[llm/start] [1:chain:SQLDatabaseChain > 2:chain:LLMChain > 3:llm:Replicate] Entering LLM run with input:
{
  "prompts": [
    "Only use the following tables:\n\nCREATE TABLE nba_roster (\n\t\"Team\" TEXT, \n\t\"NAME\" TEXT, \n\t\"Jersey\" TEXT, \n\t\"POS\" TEXT, \n\t\"AGE\" INTEGER, \n\t\"HT\" TEXT, \n\t\"WT\" TEXT, \n\t\"COLLEGE\" TEXT, \n\t\"SALARY\" TEXT\n)\n\nQuestion: Which team is Klay Tho

In [19]:
answer = db._execute(sql)
print(answer)

[{'Team': 'Golden State Warriors'}]


In [20]:
import json

memory.save_context({"input": question},
                    {"output": json.dumps(answer)})
followup = "What's his salary"
followup_answer = db_chain_memory.run(followup)
print(followup_answer)

# "Entering Chain run with input" does show "history" including our question: "Human: Which team is Klay Thompson in"
# but it doesn't get passed to "Entering LLM run with input", so the llm output still is Lebron James.

[chain/start] [1:chain:SQLDatabaseChain] Entering Chain run with input:
{
  "query": "What's his salary",
  "history": "Human: Which team is Klay Thompson in\nAI: SELECT Team FROM nba_roster WHERE NAME = 'Klay Thompson';\nHuman: Which team is Klay Thompson in\nAI: [{\"Team\": \"Golden State Warriors\"}]"
}
[chain/start] [1:chain:SQLDatabaseChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "What's his salary\nSQLQuery:",
  "top_k": "5",
  "dialect": "sqlite",
  "table_info": "\nCREATE TABLE nba_roster (\n\t\"Team\" TEXT, \n\t\"NAME\" TEXT, \n\t\"Jersey\" TEXT, \n\t\"POS\" TEXT, \n\t\"AGE\" INTEGER, \n\t\"HT\" TEXT, \n\t\"WT\" TEXT, \n\t\"COLLEGE\" TEXT, \n\t\"SALARY\" TEXT\n)",
  "stop": [
    "\nSQLResult:"
  ],
  "history": "Human: Which team is Klay Thompson in\nAI: SELECT Team FROM nba_roster WHERE NAME = 'Klay Thompson';\nHuman: Which team is Klay Thompson in\nAI: [{\"Team\": \"Golden State Warriors\"}]"
}
[llm/start] [1:chain:SQLDatabaseChain > 2:chain:LLMChain

Looks like we hit a [known issue](https://github.com/langchain-ai/langchain/issues/6918#issuecomment-1632932653) with adding memory to SQLDatabaseChain, even after the [PR](https://github.com/langchain-ai/langchain/pull/7546) that's supposed to fix it was merged. We'll leave the notebook using the experimental SQLDatabaseChain as is. To find a better solution to using SQLDatabaseChain, we can:
1. Play with the prompt engineering further;
2. Look into the SQLDatabaseChain's implementation and the use of ConversationBufferMemory to fix the memory issue;
3. Wait till SQLDatabaseChain prompted from the langchain_experimental package to langchain.